# Stratification corrections and Newton Raphson method to calculate Monin Obukhov length $z/L$

In [1]:
from sympy import symbols, init_printing
import sympy as sp

# Required only in terminal
# init_printing()

In [2]:
from sympy.printing.pycode import pycode

In [3]:
z1, z0, z = symbols('z_1 z_0 z', real=True, positive=True)
L, xi = symbols(r'L \xi', real=True)

In [4]:
z.is_positive

True

In [5]:
L.is_positive

Let's create dummy variables depending on Monin Obukhov length $L$ (because [Sympy cannot refine piecewise functions](https://github.com/sympy/sympy/issues/9384))

In [63]:
Lp = symbols('L', positive=True)
Ln = symbols('L', negative=True)

In [64]:
Lp.is_positive

True

In [65]:
Ln.is_positive

False

# Momentum

In [9]:
phi_m = sp.Piecewise((1 + 5*xi, xi >= 0), (sp.root((1-16*xi), 4), xi < 0))
phi_m

Piecewise((5*\xi + 1, \xi >= 0), ((1 - 16*\xi)**(1/4), True))

In [10]:
psi_m_int = (1 - phi_m) / xi

psi_m = sp.integrate(psi_m_int, (xi, 0, z/L))

In [11]:
psi_m

Piecewise((-4*(1 - 16*Min(0, z/L))**(1/4) - log((1 - 16*Min(0, z/L))**(1/4) - 1) + log((1 - 16*Min(0, z/L))**(1/4) + 1) + log(Min(0, z/L)) + 2*atan((1 - 16*Min(0, z/L))**(1/4)) + log(2) + 4 + pi*(-1 - 2*I)/2, z/L < 0), (5*Min(0, z/L) - 5*z/L, True))

In [12]:
psi_m.refine(sp.Q.zero(z)).subs(z, 0)

0

In [13]:
psi_m_stable = psi_m.refine(sp.Q.positive(L)).subs(L, Lp)
psi_m_stable

-5*z/L

In [14]:
psi_m_unstable = psi_m.refine(sp.Q.negative(L)).subs(L, Ln)
psi_m_unstable

-4*(1 - 16*z/L)**(1/4) + log(z/L) - log((1 - 16*z/L)**(1/4) - 1) + log((1 - 16*z/L)**(1/4) + 1) + 2*atan((1 - 16*z/L)**(1/4)) + log(2) + 4 + pi*(-1 - 2*I)/2

In [15]:
psi_m_unstable.expand()

-4*(1 - 16*z/L)**(1/4) + log(-1/L) + log(z) - log((1 - 16*z/L)**(1/4) - 1) + log((1 - 16*z/L)**(1/4) + 1) + 2*atan((1 - 16*z/L)**(1/4)) - pi/2 + log(2) + 4

In [56]:
alpha, a, x = symbols(r'\alpha a x', real=True)

In [69]:
psi_m_unstable.subs({sp.root(1 -16*z/Ln, 4): x, z/Ln: xi}).simplify()

-4*x + log(\xi) - log(x - 1) + log(x + 1) + 2*atan(x) - pi/2 + log(2) + 4 - I*pi

In [18]:
def f_psi_m_stable(x):
    return psi_m_stable.subs(z/Lp, x)


PsiM_stable  = sp.log(z1 / z0) - f_psi_m_stable(z1 / L) + f_psi_m_stable(z0 / L)
PsiM_stable

log(z_1/z_0) - 5*z_0/L + 5*z_1/L

In [19]:
print(pycode(PsiM_stable))

math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L


In [20]:
def f_psi_m_unstable(x):
    return psi_m_unstable.subs(z/Ln, x)


PsiM_unstable = sp.log(z1 / z0) - f_psi_m_unstable(z1 / L) + f_psi_m_unstable(z0 / L)
PsiM_unstable

-4*(1 - 16*z_0/L)**(1/4) + 4*(1 - 16*z_1/L)**(1/4) + log(z_0/L) - log(z_1/L) + log(z_1/z_0) - log((1 - 16*z_0/L)**(1/4) - 1) + log((1 - 16*z_0/L)**(1/4) + 1) + log((1 - 16*z_1/L)**(1/4) - 1) - log((1 - 16*z_1/L)**(1/4) + 1) + 2*atan((1 - 16*z_0/L)**(1/4)) - 2*atan((1 - 16*z_1/L)**(1/4))

In [21]:
print(pycode(PsiM_unstable))

-4*(1 - 16*z_0/L)**(1/4) + 4*(1 - 16*z_1/L)**(1/4) + math.log(z_0/L) - math.log(z_1/L) + math.log(z_1/z_0) - math.log((1 - 16*z_0/L)**(1/4) - 1) + math.log((1 - 16*z_0/L)**(1/4) + 1) + math.log((1 - 16*z_1/L)**(1/4) - 1) - math.log((1 - 16*z_1/L)**(1/4) + 1) + 2*math.atan((1 - 16*z_0/L)**(1/4)) - 2*math.atan((1 - 16*z_1/L)**(1/4))


# Heat

In [22]:
phi_h = sp.Piecewise((1 + 5*xi, xi >= 0), (sp.root((1-16*xi), 2), xi < 0))
phi_h

Piecewise((5*\xi + 1, \xi >= 0), (sqrt(1 - 16*\xi), True))

In [23]:
psi_h_int = (1 - phi_h) / xi

psi_h = sp.integrate(psi_h_int, (xi, 0, z/L))

In [24]:
psi_h

Piecewise((-2*sqrt(1 - 16*Min(0, z/L)) - log(sqrt(1 - 16*Min(0, z/L)) - 1) + log(sqrt(1 - 16*Min(0, z/L)) + 1) + log(Min(0, z/L)) + 2*log(2) + 2 - I*pi, z/L < 0), (5*Min(0, z/L) - 5*z/L, True))

In [25]:
psi_h.refine(sp.Q.zero(z)).subs(z, 0)

0

In [26]:
psi_h_stable = psi_h.refine(sp.Q.positive(L)).subs(L, Lp)
psi_h_stable

-5*z/L

In [27]:
psi_h_unstable = psi_h.refine(sp.Q.negative(L)).subs(L, Ln)
psi_h_unstable

-2*sqrt(1 - 16*z/L) + log(z/L) - log(sqrt(1 - 16*z/L) - 1) + log(sqrt(1 - 16*z/L) + 1) + 2*log(2) + 2 - I*pi

In [28]:
psi_h_unstable.expand()

-2*sqrt(1 - 16*z/L) + log(-1/L) + log(z) - log(sqrt(1 - 16*z/L) - 1) + log(sqrt(1 - 16*z/L) + 1) + 2*log(2) + 2

In [29]:
alpha, a = symbols(r'\alpha a', real=True)

In [30]:
psi_h_unstable.subs({1 -16*z/Ln: alpha, z/Ln: a})

-2*sqrt(\alpha) + log(a) - log(sqrt(\alpha) - 1) + log(sqrt(\alpha) + 1) + 2*log(2) + 2 - I*pi

In [31]:
def f_psi_h_stable(x):
    return psi_h_stable.subs(z/Lp, x)


PsiH_stable  = sp.log(z1 / z0) - f_psi_h_stable(z1 / L) + f_psi_h_stable(z0 / L)
PsiH_stable

log(z_1/z_0) - 5*z_0/L + 5*z_1/L

In [32]:
def f_psi_h_unstable(x):
    return psi_h_unstable.subs(z/Ln, x)


PsiH_unstable = sp.log(z1 / z0) - f_psi_h_unstable(z1 / L) + f_psi_h_unstable(z0 / L)
PsiH_unstable

-2*sqrt(1 - 16*z_0/L) + 2*sqrt(1 - 16*z_1/L) + log(z_0/L) - log(z_1/L) + log(z_1/z_0) - log(sqrt(1 - 16*z_0/L) - 1) + log(sqrt(1 - 16*z_0/L) + 1) + log(sqrt(1 - 16*z_1/L) - 1) - log(sqrt(1 - 16*z_1/L) + 1)

# Richardson number

In [33]:
Ri_b = symbols('Ri_b', real=True)

In [34]:
Ri_stable = (z1 / L) * (PsiH_stable / PsiM_stable**2) 

Ri_stable

z_1/(L*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L))

In [46]:
Ri_stable.diff(L).simplify()

-log((z_1/z_0)**z_1)/(-5*z_0 + 5*z_1 + log((z_1/z_0)**L))**2

In [47]:
Ri_unstable = (z1 / L) * (PsiH_unstable / PsiM_unstable**2)

Ri_unstable.simplify()

z_1*(-2*sqrt((L - 16*z_0)/L) + 2*sqrt((L - 16*z_1)/L) - log(sqrt((L - 16*z_0)/L) - 1) + log(sqrt((L - 16*z_0)/L) + 1) + log(sqrt((L - 16*z_1)/L) - 1) - log(sqrt((L - 16*z_1)/L) + 1))/(L*(-4*((L - 16*z_0)/L)**(1/4) + 4*((L - 16*z_1)/L)**(1/4) - log(((L - 16*z_0)/L)**(1/4) - 1) + log(((L - 16*z_0)/L)**(1/4) + 1) + log(((L - 16*z_1)/L)**(1/4) - 1) - log(((L - 16*z_1)/L)**(1/4) + 1) + 2*atan(((L - 16*z_0)/L)**(1/4)) - 2*atan(((L - 16*z_1)/L)**(1/4)))**2)

In [50]:
Ri_unstable.diff(L)

z_1*(-16*z_0/(L**2*sqrt(1 - 16*z_0/L)) + 8*z_0/(L**2*sqrt(1 - 16*z_0/L)*(sqrt(1 - 16*z_0/L) + 1)) - 8*z_0/(L**2*sqrt(1 - 16*z_0/L)*(sqrt(1 - 16*z_0/L) - 1)) + 16*z_1/(L**2*sqrt(1 - 16*z_1/L)) - 8*z_1/(L**2*sqrt(1 - 16*z_1/L)*(sqrt(1 - 16*z_1/L) + 1)) + 8*z_1/(L**2*sqrt(1 - 16*z_1/L)*(sqrt(1 - 16*z_1/L) - 1)))/(L*(-4*(1 - 16*z_0/L)**(1/4) + 4*(1 - 16*z_1/L)**(1/4) + log(z_0/L) - log(z_1/L) + log(z_1/z_0) - log((1 - 16*z_0/L)**(1/4) - 1) + log((1 - 16*z_0/L)**(1/4) + 1) + log((1 - 16*z_1/L)**(1/4) - 1) - log((1 - 16*z_1/L)**(1/4) + 1) + 2*atan((1 - 16*z_0/L)**(1/4)) - 2*atan((1 - 16*z_1/L)**(1/4)))**2) + z_1*(32*z_0/(L**2*(1 - 16*z_0/L)**(3/4)) - 16*z_0/(L**2*(1 - 16*z_0/L)**(3/4)*(sqrt(1 - 16*z_0/L) + 1)) - 8*z_0/(L**2*(1 - 16*z_0/L)**(3/4)*((1 - 16*z_0/L)**(1/4) + 1)) + 8*z_0/(L**2*(1 - 16*z_0/L)**(3/4)*((1 - 16*z_0/L)**(1/4) - 1)) - 32*z_1/(L**2*(1 - 16*z_1/L)**(3/4)) + 16*z_1/(L**2*(1 - 16*z_1/L)**(3/4)*(sqrt(1 - 16*z_1/L) + 1)) + 8*z_1/(L**2*(1 - 16*z_1/L)**(3/4)*((1 - 16*z_1/L)**(1

# Newton Raphson

In [38]:
from sympy.codegen.algorithms import newtons_method
from sympy.codegen.pyutils import render_as_module
# For Fortran
from sympy.codegen.futils import render_as_module as render_as_fortran_mod

In [39]:
dL, atol = symbols('dL atol')

In [40]:
f = Ri_b - Ri_stable

expr = L - f / f.diff(L)
algo = newtons_method(expr, L, atol, dL)

In [41]:
print(render_as_module(algo))

import math

dL = float('inf')
while (abs(dL) > atol):
    dL = -(-L + (Ri_b - z_1/(L*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)))/(-z_1*(-5*z_0/L**2 + 5*z_1/L**2)/(L*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**2) + z_1/(L**2*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L))))*(-z_1*(-5*z_0/L**2 + 5*z_1/L**2)/(L*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**2) + z_1/(L**2*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)))**2/((Ri_b - z_1/(L*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)))*(z_1*(10*z_0/L**3 - 10*z_1/L**3)/(L*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**2) + z_1*(-10*z_0/L**2 + 10*z_1/L**2)*(-5*z_0/L**2 + 5*z_1/L**2)/(L*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**3) - 2*z_1*(-5*z_0/L**2 + 5*z_1/L**2)/(L**2*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**2) + 2*z_1/(L**3*(math.log(z_1/z_0) - 5*z_0/L + 5*z_1/L))))
    L += dL


In [42]:
print(render_as_fortran_mod(algo, 'find_ob_len_stable'))

module find_ob_len_stable
   
   implicit none
   private
   public 


contains

real*8 :: dL = (huge(0d0) + 1)
do while (abs(dL) > atol)
   dL = -(-L + (Ri_b - z_1/(L*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L)))/(-z_1 &
      *(-5*z_0/L**2 + 5*z_1/L**2)/(L*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L) &
      **2) + z_1/(L**2*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L))))*(-z_1*(-5* &
      z_0/L**2 + 5*z_1/L**2)/(L*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**2) &
      + z_1/(L**2*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L)))**2/((Ri_b - z_1/ &
      (L*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L)))*(z_1*(10*z_0/L**3 - 10* &
      z_1/L**3)/(L*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**2) + z_1*(-10* &
      z_0/L**2 + 10*z_1/L**2)*(-5*z_0/L**2 + 5*z_1/L**2)/(L*(log(z_1/ &
      z_0) - 5*z_0/L + 5*z_1/L)**3) - 2*z_1*(-5*z_0/L**2 + 5*z_1/L**2)/ &
      (L**2*(log(z_1/z_0) - 5*z_0/L + 5*z_1/L)**2) + 2*z_1/(L**3*(log( &
      z_1/z_0) - 5*z_0/L + 5*z_1/L))))
   L = L + dL
end do
end module


In [43]:
f = Ri_b - Ri_unstable

expr = L - f / f.diff(L)
algo = newtons_method(expr, L, atol, dL)

#### Note: huge code generated

print(render_as_module(algo))

print(render_as_fortran_mod(algo, 'find_ob_len_stable'))